In [1]:
import pandas as pd
import numpy as np

In [2]:
entprise_train = pd.read_csv('train/entprise_info.csv',engine='python')
base_info = pd.read_csv('train/base_info.csv',engine='python',encoding='utf-8')
entprise_test = pd.read_csv('entprise_evaluate.csv')

In [3]:
entprise = pd.concat([entprise_train,entprise_test])
data = pd.merge(entprise,base_info,on='id')
data.drop(['ptbusscope','midpreindcode','score'],axis=1,inplace=True)

In [4]:
data['enttypeitem'] = data['enttypegb']//10*10
data['enttypeminu'] = data['enttypegb']

In [9]:
data['opform'].fillna(10,inplace=True)
data['opform'] = data['opform'].astype(str)

In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()
data['opform'] = labelencoder1.fit_transform(data['opform'])
data['oploc'] = labelencoder2.fit_transform(data['oploc'])
data['industryphy'] = labelencoder3.fit_transform(data['industryphy'])
data['oplocdistrict'] = labelencoder4.fit_transform(data['oplocdistrict'])

In [11]:
def opto_process(x):
    if len(x)>3:
        return x[:4]
    else:
        return x
data['opfrom'] = data['opfrom'].map(lambda x:x[:4])
data['opto'].fillna('nan',inplace=True)
data['opto'] = data['opto'].map(opto_process)
data['opfrom'] = data['opfrom'].astype(int)
data['opto'] = data['opto'].astype(float)

In [12]:
data['industryco'] = data['industryco'].fillna(data['industryco'].mean())

In [13]:
data.loc[data['opto'].isnull(),"opto"] = data[data['opto'].isnull()]['opfrom']+50.0

In [14]:
data['opscope'] = data['opscope'].apply(lambda x:x.replace("、","/").replace("；","/").replace("，","/").replace("。","/"))
data['opscope'] = data['opscope'].apply(lambda x:x.replace("（","/").replace("）","/").replace("，","/").replace("。","/"))
data['opscope'] = data['opscope'].apply(lambda x:"".join(x.split("/")))
import jieba
data['opscope_1'] = data['opscope'].apply(lambda x:jieba.lcut(x,cut_all=True))
from collections import Counter
def jieba_func(x):
    str1 = list(Counter(x).keys())[0]
    str2 = list(Counter(x).keys())[1]
    count = str1+str2
    return count
data['count_opscope'] = data['opscope'].apply(lambda x:jieba_func(x))
le6 = LabelEncoder()
data['opscope_length'] = data['opscope_1'].apply(lambda x:len(x))
# data['opscope'] = data['opscope'].apply(lambda x:x.split('/')[-2:])
data['opscope_label'] = le6.fit_transform(data['count_opscope'])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.550 seconds.
Prefix dict has been built successfully.


In [17]:
columns = data.columns.tolist()
for j in columns:
    columns_dtype = data[j].dtype
    if columns_dtype == 'object':
        data.drop(j,axis=1,inplace=True)
    else:
        continue

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24865 entries, 0 to 24864
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           14865 non-null  float64
 1   oplocdistrict   24865 non-null  int64  
 2   industryphy     24865 non-null  int64  
 3   industryco      24865 non-null  float64
 4   enttype         24865 non-null  int64  
 5   enttypeitem     24865 non-null  int64  
 6   opfrom          24865 non-null  int32  
 7   opto            24865 non-null  float64
 8   state           24865 non-null  int64  
 9   orgid           24865 non-null  int64  
 10  jobid           24865 non-null  int64  
 11  adbusign        24865 non-null  int64  
 12  townsign        24865 non-null  int64  
 13  regtype         24865 non-null  int64  
 14  empnum          19615 non-null  float64
 15  compform        10631 non-null  float64
 16  parnum          2339 non-null   float64
 17  exenum          1378 non-null  

In [20]:
x_train = data[data['empnum'].notnull()]
y_train = x_train['empnum']
x_train = x_train.drop("empnum",axis=1)
x_test = data[data['empnum'].isnull()]
x_test = x_test.drop('empnum',axis=1)

In [23]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19615 entries, 0 to 24861
Data columns (total 30 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   label           12502 non-null  float64
 1   oplocdistrict   19615 non-null  int64  
 2   industryphy     19615 non-null  int64  
 3   industryco      19615 non-null  float64
 4   enttype         19615 non-null  int64  
 5   enttypeitem     19615 non-null  int64  
 6   opfrom          19615 non-null  int32  
 7   opto            19615 non-null  float64
 8   state           19615 non-null  int64  
 9   orgid           19615 non-null  int64  
 10  jobid           19615 non-null  int64  
 11  adbusign        19615 non-null  int64  
 12  townsign        19615 non-null  int64  
 13  regtype         19615 non-null  int64  
 14  compform        10630 non-null  float64
 15  parnum          2327 non-null   float64
 16  exenum          1366 non-null   float64
 17  opform          19615 non-null 

In [25]:
from sklearn.metrics import auc, roc_curve,f1_score,mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
#from catboost import CatBoostRegressor
import lightgbm as lgb
# 模型
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=100,
                              learning_rate=0.08, n_estimators=50, max_depth=5, 
                              metric='rmse', bagging_fraction = 0.8, feature_fraction = 0.5)
def stat_func(x):
    l1 = []
    for i in x:
        if i>0.5:
            a = 1
            l1.append(a)
        else:
            a = 0
            l1.append(a)
    return l1
cat_list = [3]
# 本地验证
kf = KFold(n_splits=5,shuffle=True,random_state=2020)
devscore = []
tevscore = []
count = 1
for tidx,didx in kf.split(x_train.index):
    print('************************************ {} ************************************'.format(str(count)))
    count += 1
    tf = x_train.iloc[tidx]
    df = x_train.iloc[didx]
    tt = y_train.iloc[tidx]
    dt = y_train.iloc[didx]
    lgb_train = lgb.Dataset(tf,tt,free_raw_data=False)
    lgb_eval = lgb.Dataset(df,dt,reference=lgb_train,free_raw_data=False)
    model_lgb.fit(tf,tt)
    pre_train = model_lgb.predict(tf)
    score = mean_squared_error(pre_train,tt)
    tevscore.append(score)
    print("train F1：%s"%score)
    pre = model_lgb.predict(df)
    score = mean_squared_error(pre,dt)
    tevscore.append(score)
    print("Valid F1：%s"%score)
print("Train average F1：%s"%np.mean(tevscore),"Valid average F1：%s"%np.mean(devscore))

************************************ 1 ************************************
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
train F1：239.90805981184428
Valid F1：58.05924221843423
************************************ 2 ************************************
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
train F1：233.1022449334925
Valid F1：92.54658236027196
************************************ 3 ************************************
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsamp

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [26]:
lgb_pre = model_lgb.predict(x_test)

In [35]:
x_test.shape

(5250, 30)

In [29]:
lgb_pre_1 = []
for i in lgb_pre:
    lgb_pre_1.append(int(i))

In [37]:
data.loc[data['empnum'].isnull(),'empnum'] = lgb_pre_1

In [39]:
data.to_csv("empnum_predict.csv",index=False)